In [1]:
import pandas as pd 
from datetime import datetime as dt
import time
import numpy as np
from tableauscraper import TableauScraper as TS

#install library and download webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import time
from datetime import datetime as dt
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)

In [2]:
url = 'https://public.tableau.com/views/covid-bjn-opini/DashboardD?:embed=y&:toolbar=n&:embed_code_version=2&:loadOrderID=0&:display_count=y&publish=yes&:origin=viz_share_link'
source = 'lawancorona.bojonegorokab.go.id'

ts = TS()
ts.loads(url)
dashboard = ts.getWorkbook()

In [3]:
for t in dashboard.worksheets:
    print(f"worksheet name : {t.name}") #show worksheet name
    print(t.data) #show dataframe for this worksheet

worksheet name : CONFIRM
  AGG(D CONFIRM KOMULATIF)-alias AGG(D CONFIRM DIRAWAT)-alias  \
0                           5493                          291   

  AGG(D CONFIRM BARU)-alias AGG(D CONFIRM SEMBUH)-alias  \
0                        33                        4701   

  AGG(D CONFIRM MENINGGAL)-alias AGG(D CONFIRM DIRAWAT PERCENT)-alias  \
0                            501                                5.298   

  AGG(D CONFIRM SEMBUH PERCENT)-alias AGG(D CONFIRM MENINGGAL PERCENT)-alias  
0                               85.58                                  9.121  
worksheet name : Data Covid-19 Bojonegoro D
   COLLECT(Geometry)-alias Max Date-alias       KLASIFIKASI-alias  \
0      8775348922517919521     01-09-2021  33-64 kasus konfirmasi   
1      5513925699624730784     01-09-2021  17-32 kasus konfirmasi   
2     13484043269915728359     01-09-2021   9-16 kasus konfirmasi   
3     12774478402657570004     01-09-2021    5-8 kasus konfirmasi   
4     11280871640707730696     

In [4]:
data1 = dashboard.worksheets[0].data
data2 = dashboard.worksheets[2].data

In [5]:
#find time and date
haha = 'https://public.tableau.com/profile/elfan#!/vizhome/covid-bjn-opini/DashboardD'
driver.get(haha)
time.sleep(3)
content = driver.page_source
soup = BeautifulSoup(content, "html.parser")
div = soup.find('div', class_='VizDetails_linkRow__2ktbQ')
links = div.find_all('span')

tanggal = links[1].text.split()
tanggal[2] = tanggal[2][:-1]
bulan = ['januari', 'februari', 'maret', 'april', 'may', 'jun','jul','aug', 'sep', 'oktober','november','desember']
bln = bulan.index(tanggal[1].lower())+1
tanggal_update = tanggal[3]+'-'+str(bln)+'-'+tanggal[2]
dates = dt.strptime(tanggal_update, '%Y-%m-%d').strftime('%Y-%m-%d')

now = dt.now().strftime('%Y-%m-%d')

In [6]:
dates

'2021-09-01'

In [7]:
#create list
scrape_date = []
date_update = []
total_odp = []
total_positif = []
positif_sembuh = []
positif_isolasi = []
positif_dirawat = []
positif_meninggal = []
source_link = []

In [8]:
#append data to list
scrape_date.append(now)
date_update.append(dates)
total_odp.append('')
total_positif.append(data1['AGG(D CONFIRM KOMULATIF)-alias'][0])
positif_sembuh.append(data1['AGG(D CONFIRM SEMBUH)-alias'][0])
positif_isolasi.append('')
positif_dirawat.append(data1['AGG(D CONFIRM DIRAWAT)-alias'][0])
positif_meninggal.append(data1['AGG(D CONFIRM MENINGGAL)-alias'][0])
source_link.append(source)

total_pdp = sum([data2['AGG(D SUSPECT DIPANTAU)-alias'][0], data2['AGG([D PROBABLE KOMULATIF]-[D PROBABLE SEMBUH])-alias'][0]])

In [9]:
#create table
df = pd.DataFrame()
df['scrape_date'] = scrape_date
df['date_update'] = date_update
df['total_odp'] = total_odp
df['total_pdp'] = total_pdp
df['total_positif'] = total_positif
df['positif_sembuh'] = positif_sembuh
df['positif_dirawat'] = positif_dirawat
df['positif_isolasi'] = positif_isolasi
df['positif_meninggal'] = positif_meninggal
df['source_link'] = source_link
df.insert(2,column='provinsi', value='Jawa Timur')
df.insert(3,column='kabkot', value='Bojonegoro')
df.insert(4, column='kecamatan', value='')
df.insert(5,column='kelurahan', value='')
df.insert(14,column='types', value='kabkot')
df.insert(15,column='user_pic', value='Muslar')

In [10]:
df

,scrape_date,date_update,provinsi,kabkot,kecamatan,kelurahan,total_odp,total_pdp,total_positif,positif_sembuh,positif_dirawat,positif_isolasi,positif_meninggal,source_link,types,user_pic
0,2021-09-02,2021-09-01,Jawa Timur,Bojonegoro,,,,31,5493,4701,291,,501,lawancorona.bojonegorokab.go.id,kabkot,Muslar


In [11]:
import mysql.connector
from mysql.connector import errorcode
from sqlalchemy import create_engine

try:
    mydb = mysql.connector.connect(host='db-blanja2',
                                   port=3306,
                                   user='covid_user',
                                   passwd='5bb6593aa078',
                                   database='covid')
    cursor = mydb.cursor()
    
    cols1 = "`,`".join([str(i) for i in df.columns.tolist()])
# Insert DataFrame records one by one. 
    for i,row in df.iterrows():
        sql = "INSERT INTO `covid19_data` (`" +cols1 + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)" 
        cursor.execute(sql, tuple(row)) 
# the connection is not autocommitted by default, so we must commit to save our # changes 
        mydb.commit()
#         print(now)
        
except mysql.connector.Error as error:
    print("Failed to insert into MySQL table {}".format(error))

finally:
    if (mydb.is_connected()):
        cursor.close()
        mydb.close()
#         print("MySQL connection is closed")
        print('Kabupaten Bojonegoro Done ' + now)

Kabupaten Bojonegoro Done 2021-09-02


In [12]:
import pymysql.cursors
import sqlalchemy

engine = sqlalchemy.create_engine('mysql+pymysql://covid_user5bb6593aa078@db-blanja2:3306/covid')